In [1]:
import pandas as pd
import os

La tabla que limita a las demás es la de las descripciones

In [2]:
df_limitante = pd.read_excel('DescripcionesEspecies.xlsx')

#### Cargamos el resto de tablas que hacen referencia a especies, y eliminamos aquellas que no esten en la tabla anterior

Tabla de taxonomía

In [ ]:
df_taxonomia = pd.read_excel('EIDOS_taxonomia.xlsx')
df_taxonomia = df_taxonomia[df_taxonomia['taxonid'].isin(df_limitante['idtaxon'])][[
    'taxonid', 'name', 'taxonrank', 'origen', 'taxonomicgroup', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'subgenus',
       'specificepithet', 'infraspecificepithet', 'nametype'
]]
# Necesitamos renombrar la columna de taxonomia porque para SQL es una palabra reservada
df_taxonomia = df_taxonomia.rename(columns={'order': 'taxonorder'})
df_taxonomia.to_excel('Taxonomia.xlsx', index=False)
os.remove('EIDOS_taxonomia.xlsx')

Tabla de ubicaciones (cuadrículas) de especies

In [ ]:
df_ubicaciones = pd.read_csv('UBICACIONES_TODAS.csv')
df_ubicaciones = df_ubicaciones[df_ubicaciones['idtaxon'].isin(df_limitante['idtaxon'])]
df_ubicaciones = df_ubicaciones['cuadricula'] = df_ubicaciones['cuadricula'].str.strip()
df_ubicaciones = df_ubicaciones[df_ubicaciones['cuadricula'].str.len() == 7]
df_ubicaciones.to_csv('CuadriculasEspecies.csv', index=False)
os.remove('UBICACIONES_TODAS.csv')

Tabla de nombres de especies

In [ ]:
df_nombres = pd.read_excel('EIDOS_nombres.xlsx')
df_nombres = df_nombres[df_nombres['idtaxon'].isin(df_limitante['idtaxon'])]
df_nombres = df_nombres[['idtaxon', 'nombre_aceptado', 'nombre_comun', 'idioma', 'espreferente']]
df_nombres.to_csv('NombresEspecies.csv', index=False)
os.remove('EIDOS_nombres.xlsx')

También eliminamos columnas innecesarias de la tabla de Rutas

In [ ]:
df_rutas = pd.read_csv('Rutas.csv')
df_rutas = df_rutas[['ID_Ruta', 'Nombre_Ruta', 'NOM_ETAPA', 'Longitud', 'Description', 'nombre_en', 'CCAA_1', 'Provincia_1']]
df_rutas = df_rutas.rename(columns={'nombre_en': 'Nombre_Ingles', 'CCAA_1': 'CCAA', 'Provincia_1': 'Provincia', 'NOM_ETAPA': 'Nombre_Etapa', 'Description': 'Descripcion'})

def clean_description(d):
    if not isinstance(d, str):
       d = ''
    else:
       d = d[d.find('Descripción'):].split('<td>')[1].split('</')[0]
    return d

df_rutas['Descripcion'] = df_rutas['Descripcion'].apply(clean_description)
df_rutas['Longitud'] = df_rutas['Longitud'].str.replace(',', '.').astype(float)
df_rutas.to_excel('Rutas.xlsx', index=False)
os.remove('Rutas.csv')

Y quitamos los espacios en blanco de la tabla de CuadriculasRutas

In [ ]:
df_cuadriculas_rutas = pd.read_csv('CuadriculasRutas.csv')
df_cuadriculas_rutas['CUADRICULA'] = df_cuadriculas_rutas['CUADRICULA'].str.strip()
df_cuadriculas_rutas.to_csv('CuadriculasRutas.csv', index=False)

Y arreglamos la consistencia de la tabla Cuadriculas

In [24]:
# Debemos asegurarnos de que la Comunidad Autonoma es consistente con la provincia
df_cuadriculas = pd.read_csv('Cuadriculas.csv')
# Para ello, obtenemos el diccionario de provincia a Comunidad Autonoma viendo las cuadriculas que solo aparecen una vez
solo_una_vez = df_cuadriculas['CUADRICULA'].value_counts() == 1
cuadriculas_solo_una_vez = df_cuadriculas[df_cuadriculas['CUADRICULA'].apply(lambda x: solo_una_vez[x])]
ccaas = {row['Provincia']: row['CCAA'] for i, row in cuadriculas_solo_una_vez.iterrows()}
df_cuadriculas['CCAA'] = df_cuadriculas['Provincia'].apply(lambda x: ccaas[x])
df_cuadriculas = df_cuadriculas.drop_duplicates()
df_cuadriculas.to_csv('Cuadriculas.csv', index=False)